In [141]:
import pandas as pd

In [142]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as st
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [148]:
diamonds = pd.read_csv('diamonds_train.csv')
diamonds_predict = pd.read_csv('diamonds_predict.csv')

In [149]:
import math

carat_log= []
for i in diamonds['carat']:
    carat_log.append(math.log(i))
diamonds['carat_log'] = carat_log

carat_log= []
for i in diamonds_predict['carat']:
    carat_log.append(math.log(i))
diamonds_predict['carat_log'] = carat_log

In [6]:
def remove_column_zero(col):
    for i in diamonds_predict[col].index:
        if diamonds_predict[col][i]==0:
            diamonds_predict[col][i] = diamonds_predict[col].median()
remove_column_zero('x')
remove_column_zero('y')
remove_column_zero('z')


<ipython-input-6-feb683e44122>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds_predict[col][i] = diamonds_predict[col].median()


In [145]:
diamonds = diamonds.loc[(diamonds['x']>0) | (diamonds['y']>0) | (diamonds['z']>0)]

In [146]:
diamonds = diamonds.loc[~((diamonds['y'] > 20) | (diamonds['z'] > 20))]

In [150]:
diamonds['ratio_length_width'] = diamonds['x']/diamonds['y']
diamonds_predict['ratio_length_width'] = diamonds_predict['x']/diamonds_predict['y']

In [151]:
shape = []
for i in diamonds['table'].index:
    if 54<diamonds['table'][i]<57 and 61<diamonds['depth'][i]<62.5:
        shape.append('Round')
    elif 52<diamonds['table'][i]<60 and 60<diamonds['depth'][i]<68:
        shape.append('Oval')
    elif 63<diamonds['table'][i]<69 and 69<diamonds['depth'][i]<76:
        shape.append('Princess')
    elif 58<diamonds['table'][i]<63 and 58<diamonds['depth'][i]<66:
        shape.append('Cushion')
    else:
        shape.append('others')

In [152]:
diamonds['shape'] = shape

In [153]:
shape = []
for i in diamonds_predict['table'].index:
    if 54<diamonds_predict['table'][i]<57 and 61<diamonds_predict['depth'][i]<62.5:
        shape.append('Round')
    elif 52<diamonds_predict['table'][i]<60 and 60<diamonds_predict['depth'][i]<68:
        shape.append('Oval')
    elif 63<diamonds_predict['table'][i]<69 and 69<diamonds_predict['depth'][i]<76:
        shape.append('Princess')
    elif 58<diamonds_predict['table'][i]<63 and 58<diamonds_predict['depth'][i]<66:
        shape.append('Cushion')
    else:
        shape.append('others')

In [154]:
diamonds_predict['shape'] = shape

In [155]:
diamonds['cut']=diamonds['cut'].map({'Fair':0,'Good':1,'Very Good':2,'Premium':3, 'Ideal':4})
diamonds['color']=diamonds['color'].map({'J':0, 'I':1, 'H':2, 'G':3, 'F': 4, 'E': 5, 'D':6})
diamonds['clarity']=diamonds['clarity'].map({'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7})

#For this column I had to search the price of the shapes. From 4 (most expensive) to 0 (less expensive):
diamonds['shape']=diamonds['shape'].map({'Round':4, 'Oval': 1, 'Princess': 3, 'Cushion':2, 'others':0})


In [156]:
diamonds_predict['cut']=diamonds_predict['cut'].map({'Fair':0,'Good':1,'Very Good':2,'Premium':3, 'Ideal':4})
diamonds_predict['color']=diamonds_predict['color'].map({'J':0, 'I':1, 'H':2, 'G':3, 'F': 4, 'E': 5, 'D':6})
diamonds_predict['clarity']=diamonds_predict['clarity'].map({'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7})

#For this column I had to search the price of the shapes. From 4 (most expensive) to 0 (less expensive):
diamonds_predict['shape']=diamonds_predict['shape'].map({'Round':4, 'Oval': 1, 'Princess': 3, 'Cushion':2, 'others':0})

In [157]:
scaler = StandardScaler()

In [158]:
X = scaler.fit_transform(diamonds.drop(['price', 'depth', 'table', 'x', 'y', 'z', 'carat'], axis=1))
y = diamonds['price'].values

In [159]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test= train_test_split(X,y, test_size = 0.2, random_state = 42)


In [160]:
x1_train, x1_test,y1_train,y1_test= train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [161]:
import lightgbm as lgb

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_test, label=y_test)

d_train

In [162]:
params = {
    'boosting_type': 'gbdt',
    'learning_rate':0.08,
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth':1,
    'verbose':10,
    'early_stopping_round': 20
}

In [163]:
n_estimators = 115
watchlist=[d_valid]


In [164]:
model = lgb.LGBMRegressor(boosting_type = 'gbdt',
    learning_rate =0.08,
    num_leaves = 64,                    
    objective ='regression',
    metric = 'rmse',
    max_depth = 6)

In [165]:
y_test

array([2970, 3004,  838, ..., 2690, 3992, 6629])

In [166]:
model.fit(x_train, y_train);

In [167]:
y_test1 = model.predict(x_test)
y_train1 = model.predict(x1_train)

In [168]:
y_test2 = model.predict(x1_test)
y_train2 = model.predict(x1_train)

In [169]:
from sklearn.metrics import mean_squared_error

In [170]:
print(f"test error: {mean_squared_error(y_pred=y_test1, y_true=y_test, squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train1, y_true=y1_train, squared=False)}")

test error: 519.9280163759129
train error: 470.18481966005794


In [171]:
print(f"test error: {mean_squared_error(y_pred=y_test2, y_true=y1_test, squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train2, y_true=y1_train, squared=False)}")

test error: 471.99616673347435
train error: 470.18481966005794


In [172]:
from sklearn.model_selection import cross_val_score

In [173]:
scores = cross_val_score(model, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error',
                        cv=10)

In [174]:
import numpy as np
np.mean(-scores)

516.8686961445053

In [175]:
from sklearn.model_selection import RandomizedSearchCV

In [176]:
param_grid = {
    'n_estimators':[114, 115, 116], 
    'max_depth': [2, 4, 8,16],
    'num_leaves': [31, 40, 60],
    'learning_rate': [0.005, 0.1],
    'bagging_fraction': [0.70, 0.75],
    'max_bin': [128, 256],
    'feature_fraction' : [0.75, 0.8],
    'bagging_frequency' : [0.70, 0.80],
    'min_data_in_leaf': [15, 20, 25]
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   10.2s finished


[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75


RandomizedSearchCV(cv=5,
                   estimator=LGBMRegressor(learning_rate=0.08, max_depth=6,
                                           metric='rmse', num_leaves=64,
                                           objective='regression'),
                   n_iter=32, n_jobs=-1,
                   param_distributions={'bagging_fraction': [0.7, 0.75],
                                        'bagging_frequency': [0.7, 0.8],
                                        'feature_fraction': [0.75, 0.8],
                                        'learning_rate': [0.005, 0.1],
                                        'max_bin': [128, 256],
                                        'max_depth': [2, 4, 8, 16],
                                        'min_data_in_leaf': [15, 20, 25],
                                        'n_estimators': [114, 115, 116],
                                        'num_leaves': [31, 40, 60]},
                   scoring='neg_root_mean_squared_error', verbose=10)

In [177]:
grid_search.best_estimator_

LGBMRegressor(bagging_fraction=0.75, bagging_frequency=0.7,
              feature_fraction=0.75, max_bin=256, max_depth=8, metric='rmse',
              min_data_in_leaf=15, n_estimators=116, num_leaves=40,
              objective='regression')

In [178]:
X_test = scaler.transform(diamonds_predict.drop(['id', 'depth', 'table', 'x', 'y', 'z', 'carat'], axis=1).values)

In [179]:
preds = grid_search.predict(X_test)

In [180]:
grid_search.best_score_

-527.1883797944788

In [181]:
output = pd.DataFrame({'id': diamonds_predict['id'], 'price': preds})
output.to_csv("lgbm_plus7.csv", index=False)

In [191]:
param_grid = {
    'n_estimators':[114, 115, 116], 
    'max_depth': [2, 4, 8,16],
    'num_leaves': [31, 40, 60],
    'learning_rate': [0.005, 0.1],
    'bagging_fraction': [0.70, 0.75],
    'max_bin': [128, 256],
    'feature_fraction' : [0.75, 0.8],
    'bagging_frequency' : [0.70, 0.80],
    'min_data_in_leaf': [15, 20, 25]
}

In [192]:
grid_search = GridSearchCV(model, 
                           param_grid=param_grid,
                           cv=10, 
                           verbose=10, 
                           scoring='neg_root_mean_squared_error',
                          n_jobs=-1)

In [193]:
grid_search.fit(X, y)

Fitting 10 folds for each of 3456 candidates, totalling 34560 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Done 9232 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 9369 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 9506 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 9645 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 9925 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 10066 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 10209 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 10352 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 10497 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 10642 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 10789 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 10936 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 11085 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 11234 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 11385 tasks      |

[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75


GridSearchCV(cv=10,
             estimator=LGBMRegressor(learning_rate=0.08, max_depth=6,
                                     metric='rmse', num_leaves=64,
                                     objective='regression'),
             n_jobs=-1,
             param_grid={'bagging_fraction': [0.7, 0.75],
                         'bagging_frequency': [0.7, 0.8],
                         'feature_fraction': [0.75, 0.8],
                         'learning_rate': [0.005, 0.1], 'max_bin': [128, 256],
                         'max_depth': [2, 4, 8, 16],
                         'min_data_in_leaf': [15, 20, 25],
                         'n_estimators': [114, 115, 116],
                         'num_leaves': [31, 40, 60]},
             scoring='neg_root_mean_squared_error', verbose=10)

In [194]:
grid_search.best_estimator_

LGBMRegressor(bagging_fraction=0.7, bagging_frequency=0.7,
              feature_fraction=0.75, max_bin=256, max_depth=8, metric='rmse',
              min_data_in_leaf=25, n_estimators=116, num_leaves=60,
              objective='regression')

In [195]:
gbm = lgb.train(params,
                d_train,
                n_estimators,
                watchlist,
                verbose_eval=1)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000052 seconds, init for row-wise cost 0.001668 seconds
[LightGBM] [Debug] col-wise cost 0.000355 seconds, row-wise cost 0.000233 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000640 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 524
[LightGBM] [Info] Number of data points in the train set: 32364, number of used features: 6
[LightGBM] [Info] Start training from score 3927.895532
[LightGBM] [Debug] Trained a tree with leaves = 2 and max_depth = 1
[1]	valid_0's rmse: 3963.17
Training until validation scores don't improve for 20 rounds
[LightGBM] [Debug] Trained a tree with leaves = 2 and max_depth = 1
[2]	valid_0's rmse: 3893.9
[LightGBM] [Debug] Trained a tree 

/Users/diegofernandezdelahoz/miniconda3/envs/ML_env/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [132]:
X_test = scaler.transform(diamonds_predict.drop(['id', 'depth', 'table', 'x', 'y', 'z', 'carat'], axis=1).values)

In [133]:
preds = grid_search.predict(X_test)

In [134]:
grid_search.best_score_

-525.5770279426351

In [135]:
gbm_t = lgb.train(params,
                d_valid,
                n_estimators,
                watchlist,
                verbose_eval=1)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000039 seconds, init for row-wise cost 0.001013 seconds
[LightGBM] [Debug] col-wise cost 0.000111 seconds, row-wise cost 0.000138 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001261 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 479
[LightGBM] [Info] Number of data points in the train set: 8090, number of used features: 6
[LightGBM] [Info] Start training from score 3995.793201
[LightGBM] [Debug] Trained a tree with leaves = 2 and max_depth = 1
[1]	training's rmse: 3890.64
Training until validation scores don't improve for 20 rounds
[LightGBM] [Debug] Trained a tree with leaves = 2 and max_depth = 1
[2]	training's rmse: 3705.32
[LightGBM] [Debug] Trained a tree with leaves = 2 and max_depth = 1
[3]	training's rmse: 3536.11
[LightGBM] [Debug] Trained a tree with leave

/Users/diegofernandezdelahoz/miniconda3/envs/ML_env/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [139]:
grid_search.best_score_

-525.5770279426351

In [140]:
output = pd.DataFrame({'id': diamonds_predict['id'], 'price': preds})
output.to_csv("lgbm_plus6.csv", index=False)